In [ ]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

## Load data
file = "train_data.csv"
df_train = pd.read_csv(file, nrows = 40730865/30) # (~ 6 days)
# Total number or rows in train_data: 40730865 (~ 6 months)
# df_train = pd.read_csv(file)
df_train = DataFrame(df_train,columns=['custid', 'groupid', 'productid','eventid',\
                                       'currency', 'platform','timestamp', 'invest'])
days = 6

In [ ]:
## Store df_train in HDF5 format
# store = pd.HDFStore('store.h5')
# %timeit store['df_train_h5'] = df_train  # save it
# del df_train
# %timeit store['df_train_h5']  # load it
# df_train_h5

In [ ]:
#( max(df_train['timestamp']) - min(df_train['timestamp']) )/86400
df_train['timestamp'] = df_train['timestamp'] - min(df_train['timestamp'])
df_train = df_train[df_train.timestamp <= (3600*24)*days]
#( max(df_train['timestamp']) - min(df_train['timestamp']) )/86400
df_train['timestamp (day of week)'] = np.digitize(df_train['timestamp'],range(0,3600*24*days,3600*24))
df_train['timestamp (hour of day)'] = np.digitize(df_train['timestamp'],range(0,3600*24*days,3600))
df_train['timestamp (hour of day)'] = df_train['timestamp (hour of day)'].apply(lambda x: x%24)

In [ ]:
## Add the mean of the invetment (group by customer id and hour of day) and call it custid_time
group_custid_time = df_train['invest'].groupby([df_train['custid'],df_train['timestamp (hour of day)']])
df_custid_time_mean = DataFrame(group_custid_time.mean())
invest_temp = df_custid_time_mean['invest'] 

invest_mean = []
for i in range(len(invest_temp)):
    if invest_temp[i] > 15:
        invest_mean.append(15)
    else:    
        invest_mean.append(invest_temp[i])
        
df_custid_time_mean.reset_index(inplace=True) 
df_custid_time_mean = df_custid_time_mean.rename(columns=lambda x: x.replace('invest', 'custid_time')) 
df_train = pd.merge(df_train,df_custid_time_mean)

In [ ]:
## Add the mean of the invetment (group by currency) and call it currency_2
group_currency = df_train['invest'].groupby(df_train['currency'])
df_currency_mean = DataFrame(group_currency.mean())
invest_mean_temp = df_currency_mean['invest'] 

invest_mean = []
for i in xrange(len(invest_mean_temp)):
    if invest_mean_temp[i] > 15:
        invest_mean.append(15)
    else:    
        invest_mean.append(invest_mean_temp[i])
        
currency = DataFrame(df_currency_mean.index)
del df_currency_mean 
df_currency = DataFrame({'invest_mean':invest_mean})
df_currency['currency'] = currency
df_currency = df_currency[['currency','invest_mean']]

df_train = pd.merge(df_train,df_currency)
df_train = df_train.rename(columns=lambda x: x.replace('invest_mean', 'currency_2')) 

In [ ]:
## Add the mean of the invetment (group by platform) and call it platform_2
group_platform = df_train['invest'].groupby(df_train['platform'])
df_platform_mean = DataFrame(group_platform.mean())
invest_mean_temp = df_platform_mean['invest'] 

invest_mean = []
for i in xrange(len(invest_mean_temp)):
    if invest_mean_temp[i] > 15:
        invest_mean.append(15)
    else:    
        invest_mean.append(invest_mean_temp[i])
        
platform = DataFrame(df_platform_mean.index)
del df_platform_mean 
df_platform = DataFrame({'invest_mean':invest_mean})
df_platform['platform'] = platform
df_platform = df_platform[['platform','invest_mean']]

df_train = pd.merge(df_train,df_platform)
df_train = df_train.rename(columns=lambda x: x.replace('invest_mean', 'platform_2')) 